<a href="https://colab.research.google.com/github/alirezash97/Machine-Learning-Course/blob/main/Final_Project/Annotations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget 'https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/23870/1781260/compressed/train.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1611834692&Signature=sba7KapwtOseu8Fj6Yoyj1tjmhrhBCXWKvdhdOhUWlIapQjsjY6Vw8uT6L7WpLutCsA%2B5y8NN59YMso50eX6tYCai72huJ1CQtpVtr9uZPR9tWuuOYFin8%2BslEGSXVgwooUHZnjUQJppXMxfS7hi70UJqdaiAiIpPv18ILPBG39TBBuOiYgCs%2Bu2yM28dJ5sh3eHjJOElA83FsX4bL8h3PimMScR4sTLaqETj%2Bm0haY%2FjS8fdAeKuSDifxD40RmJz99UsHHkmt%2FWaKWGX5YXlVfwr8rN6pH10CZQ%2FCpQvyRvlkB1CpMiqdl8UrLzukLzdrgrAAp0PiXKwnzrwU%2B7Jw%3D%3D&response-content-disposition=attachment%3B+filename%3Dtrain.zip'



!wget 'https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/23870/1781260/compressed/train_annotations.csv.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1611897470&Signature=OteKBnljOwGhvVGphEn3gxjPtULu7rF6lCl8QhWUzknSMWdayANHoEab1p4ObT%2F7UeLmBGZ90Cp6%2B0imtPAhxAUpXzISZQvRZObQWPbUJW8SRTjJS2%2Bbp2ZOlXe%2Be0iXiUO%2FvPIULpj2EIKbBtuvByStii4jWfsk5Ju376bRHwxv7YQuDf2%2BbNoacXBQ1AfhV%2FM7XGiN6rgM%2BfVOAPHaEacXHk1vSXZcmfgs38VcP5BZVD6DQW5lVF2X%2Fm0rxfIFtCUrq0%2Btes5KnibXDifU3iY59z4%2FIAMCKhjIGXZ%2FddaSmnQaRCMkmMTJRtCASd4e6rll2AFCI1XFR5CL8Z%2FROw%3D%3D&response-content-disposition=attachment%3B+filename%3Dtrain_annotations.csv.zip'

The name is too long, 511 chars total.
Trying to shorten...
New name is train.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1611834692&Signature=sba7KapwtOseu8Fj6Yoyj1tjmhrhBCXWKvdhdOhUWlIapQjsjY6Vw8uT6L7WpLutCsA+5y8NN59YMso50eX6tYCai72huJ1CQtpVtr9uZPR9tWuuOYFin8+slEGSXVgwooUHZnj.
--2021-01-26 09:13:33--  https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/23870/1781260/compressed/train.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1611834692&Signature=sba7KapwtOseu8Fj6Yoyj1tjmhrhBCXWKvdhdOhUWlIapQjsjY6Vw8uT6L7WpLutCsA%2B5y8NN59YMso50eX6tYCai72huJ1CQtpVtr9uZPR9tWuuOYFin8%2BslEGSXVgwooUHZnjUQJppXMxfS7hi70UJqdaiAiIpPv18ILPBG39TBBuOiYgCs%2Bu2yM28dJ5sh3eHjJOElA83FsX4bL8h3PimMScR4sTLaqETj%2Bm0haY%2FjS8fdAeKuSDifxD40RmJz99UsHHkmt%2FWaKWGX5YXlVfwr8rN6pH10CZQ%2FCpQvyRvlkB1CpMiqdl8UrLzukLzdrgrAAp0PiXKwnzrwU%2B7Jw%3D%3D&response-content-disposition=attachment%3B+filename%3Dtrain.zip
Resolving storage.googleapis.com (storage.go

In [ ]:
# !mkdir /content/trainset
# !mkdir /content/trainset/data/
# !mkdir /content/trainset/data/1/
# !unzip '/content/train.zip' -d /content/trainset/data/1/
# !unzip '/content/train_annotations.csv.zip' -d /content/trainset/annotations/

In [4]:
import pandas as pd 
annotations = pd.read_csv('/content/trainset/annotations/train_annotations.csv')
annotations.head()

,StudyInstanceUID,label,data
0,1.2.826.0.1.3680043.8.498.12616281126973421762...,CVC - Normal,"[[1487, 1279], [1477, 1168], [1472, 1052], [14..."
1,1.2.826.0.1.3680043.8.498.12616281126973421762...,CVC - Normal,"[[1328, 7], [1347, 101], [1383, 193], [1400, 2..."
2,1.2.826.0.1.3680043.8.498.72921907356394389969...,CVC - Borderline,"[[801, 1207], [812, 1112], [823, 1023], [842, ..."
3,1.2.826.0.1.3680043.8.498.11697104485452001927...,CVC - Normal,"[[1366, 961], [1411, 861], [1453, 751], [1508,..."
4,1.2.826.0.1.3680043.8.498.87704688663091069148...,NGT - Normal,"[[1862, 14], [1845, 293], [1801, 869], [1716, ..."


In [29]:
import numpy as np 

sample = annotations.iloc[1, :]
img_name = sample['StudyInstanceUID']
landmarks = sample['data']
print(type(landmarks))
landmarks = np.array(list(landmarks))
print(landmarks)

# print('Image name: {}'.format(img_name))
# print('Landmarks shape: {}'.format(landmarks.shape))
# print('First 4 Landmarks: {}'.format(landmarks[:4]))

<class 'str'>
['[' '[' '1' '3' '2' '8' ',' ' ' '7' ']' ',' ' ' '[' '1' '3' '4' '7' ','
 ' ' '1' '0' '1' ']' ',' ' ' '[' '1' '3' '8' '3' ',' ' ' '1' '9' '3' ']'
 ',' ' ' '[' '1' '4' '0' '0' ',' ' ' '2' '6' '7' ']' ',' ' ' '[' '1' '4'
 '1' '1' ',' ' ' '3' '6' '6' ']' ',' ' ' '[' '1' '4' '0' '0' ',' ' ' '4'
 '2' '8' ']' ',' ' ' '[' '1' '3' '8' '7' ',' ' ' '5' '4' '5' ']' ',' ' '
 '[' '1' '3' '9' '4' ',' ' ' '6' '4' '0' ']' ',' ' ' '[' '1' '4' '0' '0'
 ',' ' ' '7' '0' '7' ']' ',' ' ' '[' '1' '4' '1' '7' ',' ' ' '7' '8' '3'
 ']' ',' ' ' '[' '1' '4' '3' '2' ',' ' ' '8' '5' '2' ']' ',' ' ' '[' '1'
 '4' '6' '2' ',' ' ' '9' '5' '3' ']' ',' ' ' '[' '1' '4' '5' '7' ',' ' '
 '1' '0' '0' '6' ']' ']']
